# Spotify Recommendation System - Experimentation Notebook

This notebook provides a complete framework for training, evaluating, and visualizing a baseline collaborative filtering model for song recommendations.

## 1. Setup

Install and import necessary libraries. We add `matplotlib` for plotting.

In [1]:
!pip install pandas scikit-learn tqdm matplotlib

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.1/8.1 MB 30.7 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 270.1/270.1 kB 17.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.8/2.8 MB 21.9 MB/s eta 0:00:00a 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.3/65.3 kB 6.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.7/4.7 MB 49.2 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 113.9/113.9 kB 17.9 MB/s eta 0:00:00

[notice] A new release of pip available: 22.3 -> 25.3
[notice] To update, run: pip install --upgrade pip


In [2]:
import pandas as pd
import numpy as np
import json
import os
from scipy.sparse import csr_matrix
from sklearn.decomposition import TruncatedSVD
from sklearn.neighbors import NearestNeighbors
from tqdm import tqdm
import matplotlib.pyplot as plt

Matplotlib is building the font cache; this may take a moment.


## 2. Experiment Tracking Setup

We'll create a list to store the results of each experiment. This will allow us to easily compare runs and plot the outcomes.

In [3]:
experiment_results = []

## 3. Core Functions (Data Loading, Evaluation)

These are the helper functions for loading data and calculating our evaluation metrics.

In [ ]:
import os
import json

# Loads one or more JSON slice files from the dataset.
# 'path' is the absolute path to the 'data' folder of the MPD.
# 'slice_nums' is a list of slice indices (e.g., [0] → playlists 0–999, [0,1] → 0–1999).
def load_playlist_slice(path, slice_nums=[0]):
    all_playlists = []
    for slice_num in slice_nums:
        filename = f'mpd.slice.{slice_num*1000}-{(slice_num+1)*1000-1}.json'
        try:
            with open(os.path.join(path, filename)) as f:
                data = json.load(f)
                all_playlists.extend(data['playlists'])
        except FileNotFoundError:
            print(f"Warning: Slice file not found at {os.path.join(path, filename)}. Skipping.")
    return all_playlists


# Precision@k:
# "Out of the top 'k' recommendations, how many were actually relevant?"
# 'recommendations' → items your model suggested.
# 'holdout_items' → actual items the user liked.
def precision_at_k(k, recommendations, holdout_items):
    recs_at_k = recommendations[:k]
    hits = len(set(recs_at_k) & set(holdout_items))
    return hits / k


# Recall@k:
# "Of all the relevant items, what proportion did we find?"
def recall_at_k(k, recommendations, holdout_items):
    recs_at_k = recommendations[:k]
    hits = len(set(recs_at_k) & set(holdout_items))
    return hits / len(holdout_items) if len(holdout_items) > 0 else 0


## 4. Experiment Runner

This is the main function that encapsulates the entire process: data loading, preprocessing, training, and evaluation. It takes your parameters, runs the experiment, and stores the results.

In [ ]:
import os
import json
import numpy as np
from scipy.sparse import csr_matrix
from sklearn.decomposition import TruncatedSVD
from sklearn.neighbors import NearestNeighbors
from tqdm import tqdm

# Main experiment function: runs one full training + evaluation cycle.
def run_experiment(slice_nums, n_components, k=10):
    print(f"--- Starting Experiment: Slices={slice_nums}, n_components={n_components}, k={k} ---")

    # --- 1. Data Loading ---
    print(f"Loading {len(slice_nums)} slice(s) of data...")
    data_path = '/Users/pabil/Downloads/spotify_million_playlist_dataset/data/'  # Adjust if needed
    playlists = load_playlist_slice(data_path, slice_nums=slice_nums)
    if not playlists:
        print("No data loaded. Aborting experiment.")
        return

    # --- 2. Data Preprocessing & Train-Test Split ---
    print("Preprocessing data and creating train-test split...")
    
    all_tracks = [track['track_uri'] for p in playlists for track in p['tracks']]
    unique_tracks = sorted(list(set(all_tracks)))

    track_to_idx = {track: i for i, track in enumerate(unique_tracks)}
    idx_to_track = {i: track for track, i in track_to_idx.items()}
    pid_to_idx = {p['pid']: i for i, p in enumerate(playlists)}

    n_playlists = len(playlists)
    n_tracks = len(unique_tracks)

    rows, cols = []
    rows, cols = [], []
    test_set = {}

    for p in playlists:
        playlist_idx = pid_to_idx[p['pid']]
        tracks = [t['track_uri'] for t in p['tracks']]

        # 80/20 train-test split for evaluation
        if len(tracks) > 5:
            np.random.shuffle(tracks)
            num_holdout = int(len(tracks) * 0.2)
            test_set[playlist_idx] = [uri for uri in tracks[-num_holdout:] if uri in track_to_idx]
            train_tracks = tracks[:-num_holdout]
        else:
            train_tracks = tracks

        for track_uri in train_tracks:
            if track_uri in track_to_idx:
                rows.append(playlist_idx)
                cols.append(track_to_idx[track_uri])

    interaction_matrix_train = csr_matrix(
        (np.ones(len(rows)), (rows, cols)),
        shape=(n_playlists, n_tracks)
    )

    # --- 3. Model Training (Truncated SVD) ---
    print(f"Training SVD model with n_components={n_components}...")
    svd = TruncatedSVD(n_components=n_components, random_state=42)
    playlist_embeddings = svd.fit_transform(interaction_matrix_train)
    track_embeddings = svd.components_.T

    # --- 4. KNN Setup ---
    knn = NearestNeighbors(n_neighbors=k * 2, metric='cosine', algorithm='brute').fit(track_embeddings)

    # Local helper function for evaluation
    def _recommend_from_playlist_for_eval(playlist_idx_for_eval, train_matrix_for_eval, n_recs_for_eval=k):
        input_track_indices = train_matrix_for_eval[playlist_idx_for_eval].indices
        if len(input_track_indices) == 0:
            return []

        playlist_vector = np.mean(track_embeddings[input_track_indices], axis=0).reshape(1, -1)

        distances, indices = knn.kneighbors(
            playlist_vector,
            n_neighbors=n_recs_for_eval + len(input_track_indices)
        )

        recommendations = []
        train_uris = {idx_to_track[i] for i in input_track_indices}

        for idx in indices.flatten():
            uri = idx_to_track[idx]
            if uri not in train_uris:
                recommendations.append(uri)

        return recommendations[:n_recs_for_eval]

    # --- 5. Evaluation ---
    print(f"Evaluating model with k={k}...")
    avg_precision = 0
    avg_recall = 0
    eval_count = 0

    for user_idx, holdout_items in tqdm(test_set.items(), desc="Evaluating"):
        if len(holdout_items) == 0:
            continue

        recommendations = _recommend_from_playlist_for_eval(user_idx, interaction_matrix_train, n_recs_for_eval=k)

        avg_precision += precision_at_k(k, recommendations, holdout_items)
        avg_recall += recall_at_k(k, recommendations, holdout_items)
        eval_count += 1

    avg_precision = avg_precision / eval_count if eval_count > 0 else 0
    avg_recall = avg_recall / eval_count if eval_count > 0 else 0

    print(f"Precision@{k}: {avg_precision:.4f}, Recall@{k}: {avg_recall:.4f}")

    # --- 6. Store Results ---
    experiment_results.append({
        'slice_nums': slice_nums,
        'data_size': len(playlists),
        'n_components': n_components,
        f'precision_at_{k}': avg_precision,
        f'recall_at_{k}': avg_recall
    })

    print("--- Experiment Complete ---\n")


## 5. Run Experiments

Now you can easily run multiple experiments by calling the `run_experiment` function with different parameters. We'll loop through a few values for `n_components` to see how it affects performance.

In [ ]:
# Clear previous results before starting a new set of experiments
experiment_results = []

# Define the set of n_components values to test
component_options = [50, 100, 150]

# --- Experiment Set 1: Train on 10,000 playlists (slices 0–9) ---
print("--- Starting experiments on 10,000 playlists (10 slices) ---")

slices_for_10k_playlists = list(range(10))  # slices 0–9

# Run experiments for each n_components value
for n_comps in component_options:
    run_experiment(slice_nums=slices_for_10k_playlists, n_components=n_comps, k=10)


# --- Experiment Set 2: Train on 100,000 playlists (slices 0–99) ---
# WARNING: This will take a very long time and require a lot of memory.
## print("\n--- Starting experiments on 100,000 playlists (100 slices) ---")

## slices_for_100k_playlists = list(range(100))  # slices 0–99

## for n_comps in component_options:
    ## run_experiment(slice_nums=slices_for_100k_playlists, n_components=n_comps, k=10)




## 6. Visualize Results

After the experiments are complete, we can convert our results list into a pandas DataFrame and plot the outcome to easily see the impact of our changes.

In [ ]:
if not experiment_results:
    print("No experiment results to plot. Please run some experiments first.")
else:
    results_df = pd.DataFrame(experiment_results)
    print("Experiment Results:")
    print(results_df)
    
    # Plotting Recall vs. n_components
    plt.figure(figsize=(10, 6))
    plt.plot(results_df['n_components'], results_df['recall_at_10'], marker='o')
    plt.title('Model Performance vs. Number of Components')
    plt.xlabel('Number of Components')
    plt.ylabel('Recall@10')
    plt.grid(True)
    plt.xticks(results_df['n_components'])
    plt.show()